## Redes Bayesianas

### Introdução
Redes Bayesianas são modelos que usam probabilidade pra representar relações entre variáveis. Elas ajudam a prever eventos com base em dados conhecidos, mesmo quando há incertezas.

Neste projeto, a ideia é aplicar redes bayesianas pra prever enchentes em São Luís. As variáveis usadas incluem precipitação nas últimas 24h, intensidade e duração da chuva, nível da maré, bairro, topografia e qualidade da drenagem Com isso, é possível calcular a probabilidade de alagamento em determinadas regiões, ajudando na prevenção e tomada de decisões.

## Setup - Instalações e Importações

Esta célula instala as bibliotecas necessárias e carrega os dados do GitHub.


---



In [19]:
!pip install pgmpy
#Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# pgmpy - Redes Bayesianas
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator



#Datasets no GitHub
url_principal = 'https://raw.githubusercontent.com/hissa02/Redes-Bayesianas/Hissa/Codigo/Dataset/sao_luis_eventos_climaticos_limpo.csv'
url_pequeno = 'https://raw.githubusercontent.com/hissa02/Redes-Bayesianas/Hissa/Codigo/Dataset/sao_luis_enchentes_limpo.csv'


##Tabela para pré-visualização
Essa tabela mostra alguns dados retirados dos Datasets


---

In [ ]:
# Carrega dataset de eventos climáticos em São Luís
df = pd.read_csv('https://raw.githubusercontent.com/hissa02/Redes-Bayesianas/Hissa/Codigo/Dataset/sao_luis_eventos_climaticos_limpo.csv')

# Carrega dataset de enchentes em São Luís
df = pd.read_csv('https://raw.githubusercontent.com/hissa02/Redes-Bayesianas/Hissa/Codigo/Dataset/sao_luis_enchentes_limpo.csv')

# Mostra 10 primeiras linhas do dataset
df.head(10)


##Grafo da rede
Este grafo mostra fatores que afetam enchentes


---



In [ ]:
# Definir a estrutura da rede
# Intensidade_Chuva → Enchente
# Mare → Enchente
# Drenagem → Enchente
model = DiscreteBayesianNetwork([
    ('Intensidade_Chuva', 'Enchente'),
    ('Mare', 'Enchente'),
    ('Drenagem', 'Enchente')
])

# Criar o grafo para visualização
plt.figure(figsize=(8, 6))

# Converter para NetworkX para plotar
G = nx.DiGraph()
G.add_edges_from(model.edges())

# Definir posições dos nós manualmente para melhor visualização
pos = {
    'Intensidade_Chuva': (0, 1),
    'Mare': (-1, 0),
    'Drenagem': (1, 0),
    'Enchente': (0, -1)
}

# Desenhar o grafo
plt.subplot(111)
nx.draw(G, pos,
        with_labels=True,
        node_color='lightblue',
        node_size=2500,
        font_size=8.5,
        font_weight='bold',
        arrows=True,
        arrowsize=20,
        arrowstyle='->',
        edge_color='gray',
        linewidths=2)

plt.title('Rede Bayesiana: Fatores que Afetam Enchentes em São Luís/MA',
          fontsize=14, fontweight='bold', pad=20)

# Adicionar legenda
plt.text(0, -1.5, 'Setas indicam relações causais',
         ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

# Mostrar informações sobre a rede
print("\nRELAÇÕES CAUSAIS:")
print("• Intensidade da Chuva → Enchente")
print("• Estado da Maré → Enchente")
print("• Sistema de Drenagem → Enchente")
print("\nA variável 'Enchente' é influenciada pelas três causas principais.")

##Tabela de Probabilidade Condicional para Enchentes

---



In [25]:
cpd_enchente = TabularCPD(
    variable='enchente',
    variable_card=2,
    values=[
        [0.95, 0.85, 0.75, 0.90, 0.70, 0.60, 0.80, 0.60, 0.40, 0.75, 0.50, 0.30],
        [0.05, 0.15, 0.25, 0.10, 0.30, 0.40, 0.20, 0.40, 0.60, 0.25, 0.50, 0.70]
    ],
    evidence=['intensidade_chuva', 'mare', 'drenagem'],
    evidence_card=[2, 3, 2],
    state_names={
        'intensidade_chuva': ['fraca', 'intensa'],
        'mare': ['baixa', 'media', 'alta'],
        'drenagem': ['adequada', 'inadequada'],
        'enchente': ['False', 'True']
    }
)

print(cpd_enchente)


+-------------------+-----+----------------------------+
| intensidade_chuva | ... | intensidade_chuva(intensa) |
+-------------------+-----+----------------------------+
| mare              | ... | mare(alta)                 |
+-------------------+-----+----------------------------+
| drenagem          | ... | drenagem(inadequada)       |
+-------------------+-----+----------------------------+
| enchente(False)   | ... | 0.3                        |
+-------------------+-----+----------------------------+
| enchente(True)    | ... | 0.7                        |
+-------------------+-----+----------------------------+
